# Driver drowsiness detection

Import the libraries required

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
import cv2
from PIL import Image

Organizing the paths

In [ ]:
# setting main path

master_path = "data_eye"

In [ ]:
# train and test path

train_path = os.path.join(master_path, "train")
test_path = os.path.join(master_path, "test")

In [ ]:
# Add test data randomly from the train data sets

import random
import shutil

# source
open_train_path = os.path.join(train_path, "open_eye")
close_train_path = os.path.join(train_path, "closed_eye")

# destination
open_test_path = os.path.join(test_path, "open_eye")
close_test_path = os.path.join(test_path, "closed_eye")

source1 = open_train_path
destination1 = open_test_path
files1 = os.listdir(source1)
no_of_files1 = len(files1) // 5

# for file_name1 in random.sample(files1, no_of_files1):
#     shutil.move(os.path.join(source1, file_name1), destination1)

source2 = close_train_path
destination2 = close_test_path
files2 = os.listdir(source2)
no_of_files2 = len(files2) // 5

# print(no_of_files1, no_of_files2)

# for file_name2 in random.sample(files2, no_of_files2):
#     shutil.move(os.path.join(source2, file_name2), destination2)

In [ ]:
# closed and open eye image paths

open_train_images = glob.glob(train_path+"\\open_eye/*.png")
closed_train_images = glob.glob(train_path+"\\closed_eye/*.png")

open_test_images = glob.glob(test_path+"\\open_eye/*.png")
closed_test_images = glob.glob(test_path+"\\closed_eye/*.png")


In [ ]:
#Creating training and test dataframes

train_list = [x for x in closed_train_images]
train_list.extend([x for x in open_train_images])

df_train = pd.DataFrame(np.concatenate([["closed_eye"]*len(closed_train_images),
                                        ["open_eye"]*len(open_train_images)]), columns=["class"])
df_train["image"] = [x for x in train_list]

test_list = [x for x in closed_test_images]
test_list.extend([x for x in open_test_images])

df_test = pd.DataFrame(np.concatenate([["closed_eye"]*len(closed_test_images),
                                       ["open_eye"]*len(open_test_images)]), columns=["class"])
df_test["image"] = [x for x in test_list]


In [ ]:
df_train

In [ ]:
plt.figure(figsize=(10,8))
plt.title("Number of cases", fontsize=12)
sns.countplot(data=df_train, x="class")

plt.show()

In [ ]:
plt.figure(figsize=(10,8))
plt.title("Number of cases", fontsize=12)
sns.countplot(data=df_test, x="class")

plt.show()

In [ ]:
# Eye images

fig, axes = plt.subplots(nrows=1, ncols=6, figsize=(15,10))

for i , ax in enumerate(axes.flat):
    img = cv2.imread(closed_train_images[i])
    img = cv2.resize(img, (512,512))
    ax.imshow(img)
    ax.set_title("Closed")
    ax.set_xticks([])
    ax.set_yticks([])
fig.tight_layout()
plt.show()

In [ ]:
# Eye images

fig, axes = plt.subplots(nrows=1, ncols=6, figsize=(15,10))

for i , ax in enumerate(axes.flat):
    img = cv2.imread(open_train_images[i])
    img = cv2.resize(img, (512,512))
    ax.imshow(img)
    ax.set_title("Open")
    ax.set_xticks([])
    ax.set_yticks([])
fig.tight_layout()
plt.show()

## Preprocessing and preparing the dataset

#Data Preparation

In [ ]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(df_train, test_size=0.2, random_state=13, stratify=df_train["class"])

In [ ]:
val_df

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(rescale=1/255)

val_datagen = ImageDataGenerator(rescale=1/255)

In [ ]:
train_generator = train_datagen.flow_from_dataframe(
                  train_df,
                  x_col="image",
                  y_col="class",
                  target_size=(150,150),
                  batch_size=32,
                  class_mode="binary",
                  seed=7)

In [ ]:
val_generator = val_datagen.flow_from_dataframe(
                val_df,
                x_col="image",
                y_col="class",
                target_size=(150,150),
                batch_size=32,
                class_mode="binary",
                seed=7)

test_generator = val_datagen.flow_from_dataframe(
                 df_test,
                 x_col="image",
                 y_col="class",
                 target_size=(150,150),
                 batch_size=32,
                 class_mode="binary",
                 shuffle=False,
                 seed=7)

#Model Architecture

In [ ]:
#importing the necessary packages

from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, BatchNormalization
from tensorflow.keras.models import Sequential

In [ ]:
#building the cnn

model = Sequential()

#convolution
# TODO
# Diagram
model.add(Conv2D(filters=16, kernel_size=(3,3), activation="relu", input_shape=(150,150,3)))

#pooling
model.add(MaxPooling2D(pool_size=(2,2)))

#2nd Conv
model.add(Conv2D(filters=32, kernel_size=(3,3), activation="relu"))

#2nd pooling
model.add(MaxPooling2D(pool_size=(2,2)))

#3rd Conv
model.add(Conv2D(filters=64, kernel_size=(3,3), activation="relu"))

#3rd Pooling
model.add(MaxPooling2D(pool_size=(2,2)))

#flatten
model.add(Flatten())

#fully connected layer
model.add(Dense(64, activation="relu"))

model.add(Dense(1, activation="sigmoid"))

In [ ]:
#compiling

model.compile(loss="binary_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])

In [ ]:
model.summary()

## Training the model

In [ ]:
model_1 = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

In [ ]:
plt.figure(figsize=(12,8))

plt.subplot(2,2,1)
plt.plot(model_1.history['loss'], label="Training Loss")
plt.plot(model_1.history["val_loss"], label="Validation Loss")
plt.legend()
plt.title("Loss Evolution")

plt.subplot(2,2,2)
plt.plot(model_1.history['accuracy'], label="Training Loss")
plt.plot(model_1.history["val_accuracy"], label="Validation Loss")
plt.legend()
plt.title("Accuracy Evolution")

## Performance Metrics

In [ ]:
evaluation = model.evaluate(test_generator)
print(f"Test Accuracy: {evaluation[1] * 100:.2f}%")

evaluation = model.evaluate(train_generator)
print(f"Train Accuracy: {evaluation[1] * 100:.2f}%")

In [ ]:
y_true = test_generator.classes
y_pred = (model.predict(test_generator) > 0.5).astype("int32")

In [ ]:
from sklearn.metrics import confusion_matrix

plt.figure(figsize=(12,5))

confusion_matrix = confusion_matrix(y_true, y_pred)
sns.heatmap(confusion_matrix, annot=True, fmt="d")

plt.xlabel("Predicted Label", fontsize=12)
plt.xlabel("True Label", fontsize=12)

plt.show()

In [ ]:
# Precision, Recall and F1-score of the model

tn, fp, fn, tp = confusion_matrix.ravel()

precision = tp/(tp+fp)
recall = tp/(tp+fn)
f1_score = (2*precision*recall/(precision+recall))

print("Recall of the model is {:.2f}".format(recall))
print("Precision of the model is {:.2f}".format(precision))
print("F1-Score: {}".format(f1_score))

# Improve the model

## Image Augmentation

In [ ]:
train_datagen_2 = ImageDataGenerator(
                        rescale=1./255,
                        shear_range=0.2,
                        zoom_range=0.2,
                        horizontal_flip=True,
                        rotation_range=10,
                        fill_mode="nearest")

val_datagen = ImageDataGenerator(
                        rescale=1./255)

In [ ]:
train_generator_2 = train_datagen_2.flow_from_dataframe(
                    train_df,
                    x_col="image",
                    y_col="class",
                    target_size=(150,150),
                    batch_size=32,
                    class_mode="binary",
                    seed=7)

val_generator = val_datagen.flow_from_dataframe(
                val_df,
                x_col="image",
                y_col="class",
                target_size=(150,150),
                batch_size=32,
                class_mode="binary",
                seed=7)

test_generator = val_datagen.flow_from_dataframe(
                 df_test,
                 x_col="image",
                 y_col="class",
                 target_size=(150,150),
                 batch_size=32,
                 class_mode="binary",
                 shuffle=False,
                 seed=7)

# Hyperparameter tuning

In [ ]:
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adam

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

reduce_lr = ReduceLROnPlateau(monitor="val_accuracy", factor=0.1, min_delta=0.0001, patience=1, verbose=1)

filepath="weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor="val_accuracy", verbose=1, save_best_only=True, mode="max")

## Network with Paddings (Extends the area of image in which the CNN acts)

In [ ]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(3,3), activation="relu", padding="same", input_shape=(150, 150, 3)))

model.add(Conv2D(filters=32, kernel_size=(3,3), activation="relu", padding="same"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=64, kernel_size=(3,3), activation="relu", padding="same"))
model.add(Conv2D(filters=64, kernel_size=(3,3), activation="relu", padding="same"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(rate=0.6))
model.add(Dense(1, activation="sigmoid"))

In [ ]:
model.compile(loss="binary_crossentropy",
              optimizer=Adam(learning_rate=0.0001),
              metrics=["accuracy"])

In [ ]:
model.summary()

In [ ]:
# train the model

htuning_model = model.fit(
                train_generator_2,
                epochs=10,
                validation_data=val_generator,
                callbacks=[reduce_lr, checkpoint])

In [ ]:
plt.figure(figsize=(12,8))

plt.subplot(2,2,1)
plt.plot(htuning_model.history['loss'], label="Training Loss")
plt.plot(htuning_model.history["val_loss"], label="Validation Loss")
plt.legend()
plt.title("Loss Evolution")

plt.subplot(2,2,2)
plt.plot(htuning_model.history['accuracy'], label="Training Loss")
plt.plot(htuning_model.history["val_accuracy"], label="Validation Loss")
plt.legend()
plt.title("Accuracy Evolution")

plt.show()

In [ ]:
evaluation = model.evaluate(test_generator)
print(f"Test Accuracy: {evaluation[1] * 100:.2f}%")

evaluation = model.evaluate(train_generator)
print(f"Train Accuracy: {evaluation[1] * 100:.2f}%")

## Confustion Matrix

In [ ]:
y_true = test_generator.classes
y_pred = (model.predict(test_generator) > 0.5).astype("int32")

In [ ]:
from sklearn.metrics import confusion_matrix

plt.figure(figsize=(12,5))

confusion_matrix = confusion_matrix(y_true, y_pred)
sns.heatmap(confusion_matrix, annot=True, fmt="d")

plt.xlabel("Predicted Label", fontsize=12)
plt.xlabel("True Label", fontsize=12)

plt.show()

In [ ]:
tn, fp, fn, tp = confusion_matrix.ravel()

precision = tp/(tp+fp)
recall = tp/(tp+fn)
f1_score = (2*precision*recall/(precision+recall))

print("Recall of the model is {:.2f}".format(recall))
print("Precision of the model is {:.2f}".format(precision))
print("F1-Score: {}".format(f1_score))

# Transfer Learning (Not really recommended)

In [ ]:
from tensorflow.keras.applications import ResNet152V2

In [ ]:
resnet_base_model = ResNet152V2(input_shape=(150,150,3), include_top=False, weights="imagenet")

In [ ]:
model_t1 = Sequential()
model_t1.add(resnet_base_model)
model_t1.add(Flatten())

model_t1.add(Dense(1024, activation="relu"))
model_t1.add(BatchNormalization())
model_t1.add(Dropout(rate=0.5))

model_t1.add(Dense(128, activation="relu"))
model_t1.add(BatchNormalization())
model_t1.add(Dropout(rate=0.4))

model_t1.add(Dense(1, activation="sigmoid"))

In [ ]:
#Freeze initial layer of the network
resnet_base_model.trainable = False

In [ ]:
model_t1.compile(loss="binary_crossentropy",
                 optimizer=Adam(learning_rate=0.001),
                 metrics=["accuracy"])

In [ ]:
model_t1_final = model_t1.fit(
                 train_generator_2,
                 epochs=10,
                 validation_data=val_generator,
                 callbacks=[reduce_lr, checkpoint])



In [ ]:
plt.figure(figsize=(12,8))

plt.subplot(2,2,1)
plt.plot(model_t1_final.history['loss'], label="Training Loss")
plt.plot(model_t1_final.history["val_loss"], label="Validation Loss")
plt.legend()
plt.title("Loss Evolution")

plt.subplot(2,2,2)
plt.plot(model_t1_final.history['accuracy'], label="Training Loss")
plt.plot(model_t1_final.history["val_accuracy"], label="Validation Loss")
plt.legend()
plt.title("Accuracy Evolution")

plt.show()

In [ ]:
evaluation = model_t1.evaluate(test_generator)
print(f"Test Accuracy: {evaluation[1] * 100:.2f}%")

evaluation = model_t1.evaluate(train_generator)
print(f"Train Accuracy: {evaluation[1] * 100:.2f}%")

In [ ]:
y_true = test_generator.classes
y_pred = (model_t1.predict(test_generator) > 0.5).astype("int32")

In [ ]:
from sklearn.metrics import confusion_matrix

plt.figure(figsize=(12,5))

confusion_matrix = confusion_matrix(y_true, y_pred)
sns.heatmap(confusion_matrix, annot=True, fmt="d")

plt.xlabel("Predicted Label", fontsize=12)
plt.xlabel("True Label", fontsize=12)

plt.show()

In [ ]:
tn, fp, fn, tp = confusion_matrix.ravel()

precision = tp/(tp+fp)
recall = tp/(tp+fn)
f1_score = (2*precision*recall/(precision+recall))

print("Recall of the model is {:.2f}".format(recall))
print("Precision of the model is {:.2f}".format(precision))
print("F1-Score: {}".format(f1_score))